In [0]:
from __future__ import absolute_import, division, print_function

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Reshape, Flatten, Dropout
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.python.keras import utils
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger
from tensorflow.keras.optimizers import Adam
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import random
import os
import sys
import cv2
import numpy as np 
import pandas as pd 


In [0]:
#!pip3 install --upgrade pip
#!pip3 install --upgrade tensorflow

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def classestoint(label):
    label = label.strip()
    if label == "Black-grass":  return 0
    if label == "Charlock":  return 1
    if label == "Cleavers":  return 2
    if label == "Common Chickweed":  return 3
    if label == "Common wheat":  return 4
    if label == "Fat Hen":  return 5
    if label == "Loose Silky-bent": return 6
    if label == "Maize":  return 7
    if label == "Scentless Mayweed": return 8
    if label == "Shepherds Purse": return 9
    if label == "Small-flowered Cranesbill": return 10
    if label == "Sugar beet": return 11
    print("Invalid Label", label)
    return 12

In [0]:
def inttoclasses(i):
    if i == 0: return "Black-grass"
    elif i == 1: return "Charlock"
    elif i == 2: return "Cleavers"
    elif i == 3: return "Common Chickweed"
    elif i == 4: return "Common wheat"
    elif i == 5: return "Fat Hen"
    elif i == 6: return "Loose Silky-bent"
    elif i == 7: return "Maize"
    elif i == 8: return "Scentless Mayweed"
    elif i == 9: return "Shepherds Purse"
    elif i == 10: return "Small-flowered Cranesbill"
    elif i == 11: return "Sugar beet"
    print("Invalid class ", i)
    return "Invalid Class"

In [0]:
train_dir="/content/gdrive/My Drive/GreatlakesPython16112018/AIML/plant-seedlings-classification/train"
test_dir="/content/gdrive/My Drive/GreatlakesPython16112018/AIML/plant-seedlings-classification/test"

In [0]:
#1.Read the images and generate the train and test dataset (5 points)
NUM_CLASSES = 12
# we need images of same size so we convert them into the size
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (WIDTH, HEIGHT, DEPTH)
# initialize number of epochs to train for, initial learning rate and batch size
EPOCHS = 15
INIT_LR = 1e-3
BS = 32

def readTrainData(train_dir):
    data = []
    labels = []
    # loop over the input images
    dirs = os.listdir(train_dir) 
    for dir in dirs:
        absDirPath = os.path.join(os.path.sep,train_dir, dir)
        images = os.listdir(absDirPath)
        for imageFileName in images:
            # load the image, pre-process it, and store it in the data list
            imageFullPath = os.path.join(train_dir, dir, imageFileName)
            img = load_img(imageFullPath)
            arr = img_to_array(img)  # Numpy array with shape (233,233,3)
            arr = cv2.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
            #print(arr.shape) 
            data.append(arr)
            label = classestoint(dir)
            labels.append(label)
    return data, labels

In [0]:
def createModel():
    model = Sequential()
    # first set of CONV => RELU => POOL layers
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
    model.add(Conv2D(32, (3,3), input_shape=inputShape))
    #model.add(BatchNormalization()) 
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(64, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(128, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(units=500))
    model.add(Activation("relu"))
    model.add(Dropout(0.1))
    model.add(Dense(units=12))
    model.add(Activation("softmax"))
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [10]:
#2.Divide the data set into Train and validation data sets
random.seed(10)
#allLabels =  os.listdir("train_dir")  # list of subdirectories and files
print("Loading images...")
sys.stdout.flush()
X, Y = readTrainData(train_dir)
# scale the raw pixel intensities to the range [0, 1]
X = np.array(X, dtype="float") / 255.0
Y = np.array(Y)
# convert the labels from integers to vectors
Y =  to_categorical(Y, num_classes=12)

print("Partition data into 90:10...")
sys.stdout.flush()
# partition the data into training and testing splits using 90% training and 10% for validation
(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.1, random_state=10)

Loading images...
Partition data into 90:10...


In [0]:
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [26]:
#3. Initialize & build the model (10 points)
#construct the image generator for data augmentation
model = KerasClassifier(build_fn=createModel, verbose=0)
print("Generating images...")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
# initialize the model
print("compiling model...")
sys.stdout.flush()
model = createModel()
# train the network
print("training network...")
sys.stdout.flush()
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), \
    validation_data=(valX, valY), \
    steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)

# save the model to disk
print("Saving model to drive")
sys.stdout.flush()
model.save("/content/gdrive/My Drive/plant-seedlings-classification/CNNmodel2")

Generating images...
compiling model...
training network...
Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 134 steps, validate on 479 samples
Epoch 1/15
134/134 [==============================] - 21s 159ms/step - loss: 2.2268 - accuracy: 0.2269 - val_loss: 1.7338 - val_accuracy: 0.3862
Epoch 2/15
134/134 [==============================] - 14s 106ms/step - loss: 1.6249 - accuracy: 0.4164 - val_loss: 1.2576 - val_accuracy: 0.5908
Epoch 3/15
134/134 [==============================] - 14s 108ms/step - loss: 1.3167 - accuracy: 0.5380 - val_loss: 1.1359 - val_accuracy: 0.6075
Epoch 4/15
134/134 [==============================] - 14s 108ms/step - loss: 1.1253 - accuracy: 0.6126 - val_loss: 0.9716 - val_accuracy: 0.6952
Epoch 5/15
134/134 [==============================] - 15s 109ms/step - loss: 1.0044 - accuracy: 0.6587 - val_loss: 0.8125 - val_accuracy: 0.7182
Epoch 6/15
134/134 [==============================] - 14s 108ms/step - loss: 0.8659 - accuracy: 0.7046 - val_loss: 0.9179 - val_accuracy: 0.6785
Epoch 7/15
134/134 [==============================] - 14s 107ms/step - loss: 0.8174 -

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/plant-seedlings-classification/CNNmodel2/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/plant-seedlings-classification/CNNmodel2/assets


In [27]:

model.evaluate(trainX,trainY)

4307/4307 [==============================] - 1s 314us/sample - loss: 0.3155 - accuracy: 0.8823


[0.31548085732639375, 0.88228464]

In [28]:
model.evaluate(valX,valY)

479/479 [==============================] - 0s 337us/sample - loss: 0.4322 - accuracy: 0.8434


[0.43218297749322243, 0.8434238]

In [29]:
y_pred = model.predict(valX)
y_class = np.argmax(y_pred, axis = 1) 
y_check = np.argmax(valY, axis = 1) 

cmatrix = confusion_matrix(y_check, y_class)
print(cmatrix)

[[12  0  0  0  1  3 11  0  0  0  0  0]
 [ 0 38  1  0  0  1  0  1  0  0  2  1]
 [ 0  1 22  0  1  1  0  0  1  0  1  0]
 [ 0  0  1 54  0  0  0  0  1  0  0  1]
 [ 2  0  0  0 17  0  0  0  0  0  0  0]
 [ 0  0  0  1  0 46  1  1  0  1  1  2]
 [15  0  0  0  0  1 50  0  1  0  0  0]
 [ 0  0  0  0  0  0  0 19  0  0  0  0]
 [ 4  0  0  1  0  0  0  0 51  1  0  1]
 [ 0  0  0  2  1  0  0  0  3 11  0  0]
 [ 0  1  0  0  0  0  0  2  0  0 48  0]
 [ 0  0  1  0  1  0  0  2  0  0  0 36]]


In [0]:
 def readTestData(test_dir):
    data = []
    filenames = []
    # loop over the input images
    images = os.listdir(test_dir)
    for imageFileName in images:
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(test_dir, imageFileName)
        #print(imageFullPath)
        img = load_img(imageFullPath)
        arr = img_to_array(img)  # Numpy array with shape (...,..,3)
        arr = cv2.resize(arr, (HEIGHT,WIDTH)) 
        data.append(arr)
        filenames.append(imageFileName)
    return data, filenames

In [31]:
# read test data and find its classification
testX, filenames = readTestData(test_dir)
# scale the raw pixel intensities to the range [0, 1]
testX = np.array(testX, dtype="float") / 255.0

from tensorflow.keras.models import load_model
mymodel = load_model('/content/gdrive/My Drive/plant-seedlings-classification/CNNmodel2')
yFit = mymodel.predict(testX, batch_size=10, verbose=1)
print(yFit) 
print(type(yFit)) # numpy.ndarray
print(type(filenames)) # list

import csv  
with open('/content/gdrive/My Drive/plant-seedlings-classification/output.csv', 'w', newline='') as csvfile:
    fieldnames = ['file', 'species']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, file in enumerate(filenames):
        classesProbs = yFit[index]
        maxIdx = 0
        maxProb = 0;
        for idx in range(0,11):
            if(classesProbs[idx] > maxProb):
                maxIdx = idx
                maxProb = classesProbs[idx]
        writer.writerow({'file': file, 'species': inttoclasses(maxIdx)})
print("Writing complete")

803/803 [==============================] - 0s 500us/sample
[[4.51067194e-07 2.71980093e-06 1.54198042e-05 ... 2.10523795e-04
  9.97199178e-01 1.34445843e-03]
 [4.46475060e-05 2.19026138e-03 3.34457546e-01 ... 4.40017584e-06
  9.88521697e-06 6.46371841e-01]
 [1.53799547e-07 1.20065236e-09 1.70484514e-12 ... 1.37563029e-06
  2.23425695e-10 2.14625260e-07]
 ...
 [1.91536937e-02 8.61917382e-09 2.98625731e-04 ... 4.18353442e-11
  3.36046675e-08 2.55432724e-06]
 [2.12368650e-10 9.68420208e-01 3.03717721e-02 ... 1.14208315e-05
  1.11223594e-03 2.38818193e-05]
 [2.96728399e-06 2.45801033e-03 1.80157344e-03 ... 4.74299986e-06
  1.99434135e-06 9.94423091e-01]]
<class 'numpy.ndarray'>
<class 'list'>
Writing complete


In [0]:
#output.csv gives 83.74% score

In [0]:
def Model():
    model = Sequential()
    # first set of CONV => RELU => POOL layers
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
    model.add(Conv2D(32, (3,3), input_shape=inputShape))
    #model.add(BatchNormalization()) 
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(64, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=500))
    model.add(Activation("relu"))
    model.add(Dense(units=12))
    model.add(Activation("softmax"))
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [34]:
#4. Optimize the model (8 points)

#construct the image generator for data augmentation
model = KerasClassifier(build_fn=Model, verbose=0)
print("Generating images...")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
# initialize the model
print("compiling model...")
sys.stdout.flush()
model = Model()
# train the network
print("training network...")
sys.stdout.flush()
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), \
    validation_data=(valX, valY), \
    steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)

# save the model to disk
print("Saving model to drive")
sys.stdout.flush()
model.save("/content/gdrive/My Drive/plant-seedlings-classification/CNNmodel_opt")

Generating images...
compiling model...
training network...
  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 134 steps, validate on 479 samples
Epoch 1/15
134/134 [==============================] - 14s 107ms/step - loss: 2.0013 - accuracy: 0.3125 - val_loss: 1.4846 - val_accuracy: 0.5073
Epoch 2/15
134/134 [==============================] - 14s 104ms/step - loss: 1.4519 - accuracy: 0.4978 - val_loss: 1.1070 - val_accuracy: 0.6701
Epoch 3/15
134/134 [==============================] - 14s 105ms/step - loss: 1.2105 - accuracy: 0.5843 - val_loss: 1.0730 - val_accuracy: 0.6263
Epoch 4/15
134/134 [==============================] - 14s 105ms/step - loss: 1.0352 - accuracy: 0.6510 - val_loss: 0.7721 - val_accuracy: 0.7578
Epoch 5/15
134/134 [==============================] - 14s 105ms/step - loss: 0.8995 - accuracy: 0.6980 - val_loss: 0.7265 - val_accuracy: 0.7766
Epoch 6/15
134/134 [==============================] - 14s 105ms/step - loss: 0.8115 - accuracy: 0.7284 - val_loss: 0.6252 - val_accuracy: 0.7683
Epoch 7/15
134/134 [==============================] - 14s 104ms/step - loss: 0.7276 -

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/plant-seedlings-classification/CNNmodel_opt/assets


In [35]:
#5. Predict the accuracy for both train and validation data (7 points)
model.evaluate(trainX,trainY)

4307/4307 [==============================] - 1s 308us/sample - loss: 0.2826 - accuracy: 0.8964


[0.28258022080587075, 0.89644766]

In [36]:
model.evaluate(valX,valY)

479/479 [==============================] - 0s 315us/sample - loss: 0.3811 - accuracy: 0.8643


[0.38108332620035384, 0.8643006]

In [37]:
y_pred = model.predict(valX)
y_class = np.argmax(y_pred, axis = 1) 
y_check = np.argmax(valY, axis = 1) 

cmatrix = confusion_matrix(y_check, y_class)
print(cmatrix)

[[ 8  0  0  0  2  0 15  0  2  0  0  0]
 [ 0 41  1  0  0  2  0  0  0  0  0  0]
 [ 0  3 22  0  0  2  0  0  0  0  0  0]
 [ 0  0  0 49  0  0  0  0  2  4  1  1]
 [ 0  0  0  0 19  0  0  0  0  0  0  0]
 [ 0  3  0  1  0 49  0  0  0  0  0  0]
 [10  0  0  0  1  3 53  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 19  0  0  0  0]
 [ 1  0  0  0  0  0  0  0 55  2  0  0]
 [ 0  0  0  0  0  0  0  0  2 15  0  0]
 [ 0  1  0  0  0  0  0  1  0  0 49  0]
 [ 0  1  1  0  2  0  0  1  0  0  0 35]]


In [38]:
# read test data and find its classification
testX, filenames = readTestData(test_dir)
# scale the raw pixel intensities to the range [0, 1]
testX = np.array(testX, dtype="float") / 255.0

mymodel = load_model('/content/gdrive/My Drive/plant-seedlings-classification/CNNmodel_opt')
yFit = mymodel.predict(testX, batch_size=10, verbose=1)
print(yFit) 
print(type(yFit)) # numpy.ndarray
print(type(filenames)) # list

import csv  
with open('/content/gdrive/My Drive/plant-seedlings-classification/output_opt.csv', 'w', newline='') as csvfile:
    fieldnames = ['file', 'species']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, file in enumerate(filenames):
        classesProbs = yFit[index]
        maxIdx = 0
        maxProb = 0;
        for idx in range(0,11):
            if(classesProbs[idx] > maxProb):
                maxIdx = idx
                maxProb = classesProbs[idx]
        writer.writerow({'file': file, 'species': inttoclasses(maxIdx)})
print("Writing complete")

803/803 [==============================] - 0s 439us/sample
[[1.3980251e-06 9.1038282e-06 4.3932501e-05 ... 1.6125438e-03
  9.9128145e-01 5.4049678e-04]
 [7.1336013e-05 1.8153027e-02 2.2554061e-01 ... 8.8442393e-06
  3.0454841e-05 7.3736489e-01]
 [3.3971727e-08 3.1150378e-08 6.6404715e-10 ... 8.9273199e-05
  1.1467404e-09 1.0123879e-05]
 ...
 [3.6933343e-03 8.6262162e-09 5.1945392e-03 ... 2.8926053e-11
  1.6969562e-07 1.0731575e-07]
 [9.4486557e-11 9.9547261e-01 4.3043843e-03 ... 1.5502539e-05
  1.4292472e-04 2.4522963e-06]
 [1.8020959e-06 8.3979107e-03 3.0307227e-03 ... 2.2125347e-05
  1.3609038e-05 9.8274833e-01]]
<class 'numpy.ndarray'>
<class 'list'>
Writing complete


In [0]:
#output.csv gives 85.17% score